In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from zipfile import ZipFile
import pandas as pd
import torch
import random
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm, trange

import sys
sys.path.append('..')

from HP import PointProcessStorage, DirichletMixtureModel, EM_clustering
from metrics import consistency, purity
from Cohortney.data_utils import load_data

### IPTV Data

In [3]:
path = Path('../..', 'data', 'IPTV_Data')
ss, Ts, class2idx, _ = load_data(path, ext='txt', datetime=True, maxlen=500)

In [4]:
N = len(ss)

In [5]:
D = 5
basis_fs = [lambda x: torch.exp(- x**2 / (2.*(k+1)**2) ) for k in range(D)]

In [6]:
C = len(class2idx)
K = 6

In [7]:
ntrials = 5
niter = 10

labels = torch.zeros(ntrials, len(ss))
nlls = torch.zeros(ntrials, niter)

for i in trange(ntrials):
    Sigma = torch.rand(C, C, D, K)
    B = torch.rand(C, K)
    alpha = 1.

    train_ids = np.sort(np.random.choice(np.arange(len(ss)), size=len(ss) // 2, replace=False))
    train_fold = [ss[i] for i in range(len(ss)) if i in train_ids]
    train_Ts = Ts[train_ids]
    
    # learn
    hp = PointProcessStorage(train_fold, train_Ts, basis_fs)
    model = DirichletMixtureModel(K, C, D, alpha, B, Sigma)
    EM = EM_clustering(hp, model, n_inner=5)

    _, nll_history, _ = EM.learn_hp(niter=niter, ninner=[2,3,4,5,6,7] + (niter-6)*[8])

    # validate
    EM.hp = PointProcessStorage(ss, Ts, basis_fs)
    EM.N = len(ss)
    EM.int_g = []
    EM.g = []
    r = EM.e_step()
    
    labels[i] = r.argmax(-1)
    nlls[i] = torch.FloatTensor(nll_history)

    print(f'\nConsistency of clustering: {consistency(labels[:i+1]).item():.4f}')

  0%|          | 0/4 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:45, 45.77s/it]
NLL / N: 590.3496

2it [01:12, 40.17s/it]
NLL / N: 601.5843

3it [01:52, 39.87s/it]
NLL / N: 612.4089

4it [02:33, 40.28s/it]
NLL / N: 667.2936

5it [03:19, 39.92s/it]
NLL / N: 704.0333

 25%|██▌       | 1/4 [04:13<12:39, 253.20s/it]
0it [00:00, ?it/s]
Consistency of clustering: nan

1it [00:39, 39.29s/it]
NLL / N: 559.0773

2it [01:19, 39.53s/it]
NLL / N: 543.5961

3it [02:18, 45.50s/it]
NLL / N: 542.3719

4it [03:27, 52.40s/it]
NLL / N: 605.5701

5it [04:38, 55.79s/it]
NLL / N: 620.2062

 50%|█████     | 2/4 [09:44<09:13, 276.65s/it]
0it [00:00, ?it/s]
Consistency of clustering: 0.5670

1it [00:38, 38.33s/it]
NLL / N: 599.6569

2it [01:13, 37.23s/it]
NLL / N: 622.2506

3it [02:00, 40.17s/it]
NLL / N: 645.7490

4it [02:52, 43.86s/it]
NLL / N: 652.4546

5it [03:48, 45.65s/it]
NLL / N: 602.4416

 75%|███████▌  | 3/4 [14:49<04:45, 285.27s/it]
0it [00:00, ?it/s]
Consistency of clustering: 0.4139

1it [00:59, 5

In [8]:
assert (model.A >= 0).all()
assert (model.mu > 0).all()

In [9]:
# plt.figure(figsize=(9, 5))
# plt.grid()
# plt.plot(np.arange(niter)+1, nlls.mean(0).numpy() / len(train_ids))
# plt.fill_between(np.arange(niter)+1, (nlls.mean(0).numpy() - nlls.std(0).numpy()) / len(train_ids), (nlls.mean(0).numpy() + nlls.std(0).numpy()) / len(train_ids), alpha=0.2)
# plt.title('Mixing DMMHP', fontsize=15)
# plt.xlabel(r'$n$ outer iterations', fontsize=15)
# plt.ylabel(r'$\sim$ NLL / $N$', fontsize=15)
# plt.show()

In [10]:
print(f'Consistency of clustering: {consistency(labels).item():.4f}')

Consistency of clustering: 0.3817


### Synthetic data

#### 5 cluster, 5 classes

In [17]:
path = Path('../..', 'data', 'simulated_Hawkes', 'K5_C5')
ss, Ts, class2idx, _ = load_data(path, maxlen=-1, ext='csv', datetime=False)

In [20]:
gt_ids = pd.read_csv(Path(path, 'clusters.csv'))['cluster_id'].to_numpy()

In [21]:
gt_ids = torch.LongTensor(gt_ids)

In [22]:
N = len(ss)
D = 5
basis_fs = [lambda x: torch.exp(- x**2 / (2.*(k+1)**2) ) for k in range(D)]

hp = PointProcessStorage(ss, Ts, basis_fs)

C = len(class2idx)
K = 5


In [23]:
ntrials = 3
niter = 10

labels = torch.zeros(ntrials, len(ss))
nlls = torch.zeros(ntrials, niter)

for i in range(ntrials):
    Sigma = torch.rand(C, C, D, K)
    B = torch.rand(C, K)
    alpha = 1.

    model = DirichletMixtureModel(K, C, D, alpha, B, Sigma)
    EM = EM_clustering(hp, model)
    r, nll_history, r_history = EM.learn_hp(niter=niter, ninner=[2,3,4,5,6,7] + (niter - 6)*[8])

    labels[i] = r.argmax(-1)
    nlls[i] = torch.FloatTensor(nll_history)

    print(f'Purity: {purity(labels[i], gt_ids):.4f}')


1it [00:48, 48.78s/it]
NLL / N: 122.0036
2it [00:59, 37.37s/it]
NLL / N: 123.2251
3it [01:14, 30.54s/it]
NLL / N: 121.3704
4it [01:28, 25.73s/it]
NLL / N: 115.8717
5it [01:47, 23.74s/it]
NLL / N: 113.1904
6it [02:10, 23.33s/it]
NLL / N: 111.8429
7it [02:37, 24.44s/it]
NLL / N: 113.9309
8it [03:08, 26.50s/it]
NLL / N: 114.1542
9it [03:41, 28.58s/it]
NLL / N: 114.0441
10it [04:14, 25.48s/it]
0it [00:00, ?it/s]
NLL / N: 113.8574
Purity: 0.5300
1it [00:51, 51.39s/it]
NLL / N: 116.7516
2it [01:03, 39.49s/it]
NLL / N: 111.9592
3it [01:17, 31.92s/it]
NLL / N: 113.1661
4it [01:31, 26.63s/it]
NLL / N: 116.1907
5it [01:50, 24.43s/it]
NLL / N: 117.9640
6it [02:13, 23.96s/it]
NLL / N: 115.9253
7it [02:41, 25.04s/it]
NLL / N: 114.1579
8it [03:12, 26.91s/it]
NLL / N: 113.6476
9it [03:46, 29.04s/it]
NLL / N: 113.5558
10it [04:23, 26.37s/it]
0it [00:00, ?it/s]
NLL / N: 113.5510
Purity: 0.6355
1it [00:53, 53.79s/it]
NLL / N: 117.0573
2it [01:09, 42.37s/it]
NLL / N: 104.5444
3it [01:25, 34.45s/it]
NLL /

In [24]:
assert (model.A >= 0).all()
assert (model.mu > 0).all()

In [25]:
# plt.figure(figsize=(9, 5))
# plt.grid()
# plt.plot(np.arange(niter)+1, nlls.numpy() / len(ss))
# plt.title('Mixing of DMMHP', fontsize=15)
# plt.xlabel(r'$n$ outer iterations', fontsize=15)
# plt.ylabel(r'$\sim$ NLL / $N$', fontsize=15)
# plt.show()

In [26]:
print([purity(x, gt_ids) for x in r_history.argmax(-1)])

[0.3245, 0.36500000000000005, 0.4465, 0.478, 0.525, 0.5830000000000001, 0.6220000000000001, 0.637, 0.6405000000000001, 0.6385]


In [27]:
pur_val_mean = np.mean([purity(x, gt_ids) for x in labels])
pur_val_std = np.std([purity(x, gt_ids) for x in labels])

In [28]:
print(f'Purity: {pur_val_mean:.4f}+-{pur_val_std:.4f}')

Purity: 0.6013+-0.0505


In [29]:
labels[::10]

tensor([[1., 1., 2.,  ..., 2., 2., 4.]])

#### 4 cluster, 5 classes

In [30]:
path = Path('../..', 'data', 'simulated_Hawkes', 'K4_C5')
ss, Ts, class2idx, _ = load_data(path, maxlen=-1, ext='csv', datetime=False)

gt_ids = pd.read_csv(Path(path, 'clusters.csv'))['cluster_id'].to_numpy()

gt_ids = torch.LongTensor(gt_ids)

In [31]:
N = len(ss)
D = 5
basis_fs = [lambda x: torch.exp(- x**2 / (2.*(k+1)**2) ) for k in range(D)]

hp = PointProcessStorage(ss, Ts, basis_fs)

C = len(class2idx)
K = 4

ntrials = 3
niter = 10

labels = torch.zeros(ntrials, len(ss))
nlls = torch.zeros(ntrials, niter)

for i in range(ntrials):
    Sigma = torch.rand(C, C, D, K)
    B = torch.rand(C, K)
    alpha = 1.

    model = DirichletMixtureModel(K, C, D, alpha, B, Sigma)
    EM = EM_clustering(hp, model)
    r, nll_history, r_history = EM.learn_hp(niter=niter, ninner=[2,3,4,5,6,7] + (niter - 6)*[8])

    labels[i] = r.argmax(-1)
    nlls[i] = torch.FloatTensor(nll_history)

    print(f'Purity: {purity(labels[i], gt_ids):.4f}')


1it [00:38, 38.65s/it]
NLL / N: 114.6463
2it [00:47, 29.58s/it]
NLL / N: 116.0253
3it [00:58, 24.29s/it]
NLL / N: 112.0306
4it [01:11, 20.63s/it]
NLL / N: 112.9056
5it [01:26, 19.18s/it]
NLL / N: 113.4149
6it [01:42, 18.13s/it]
NLL / N: 117.1065
7it [02:00, 17.99s/it]
NLL / N: 117.4913
8it [02:19, 18.42s/it]
NLL / N: 117.1440
9it [02:47, 21.24s/it]
NLL / N: 116.8707
10it [03:23, 25.82s/it]
NLL / N: 117.1514
11it [04:06, 30.75s/it]
NLL / N: 117.7376
12it [04:39, 31.50s/it]
NLL / N: 118.0698
13it [05:05, 29.72s/it]
NLL / N: 118.3414
14it [05:33, 29.27s/it]
NLL / N: 118.4408
15it [05:57, 27.70s/it]
NLL / N: 118.5149
16it [06:20, 26.35s/it]
NLL / N: 118.5729
17it [06:43, 25.38s/it]
NLL / N: 118.6344
18it [07:06, 24.58s/it]
NLL / N: 118.6989
19it [07:30, 24.44s/it]
NLL / N: 118.7644
20it [08:05, 24.29s/it]
0it [00:00, ?it/s]
NLL / N: 118.8272
Purity: 0.7450
1it [00:55, 55.74s/it]
NLL / N: 116.5196
2it [01:06, 42.34s/it]
NLL / N: 115.5302
3it [01:22, 34.39s/it]
NLL / N: 121.5732
4it [01:38, 

In [32]:
pur_val_mean = np.mean([purity(x, gt_ids) for x in labels])
pur_val_std = np.std([purity(x, gt_ids) for x in labels])

In [33]:
print(f'Purity: {pur_val_mean:.4f}+-{pur_val_std:.4f}')

Purity: 0.7395+-0.0703


#### 3 cluster, 5 classes

In [35]:
path = Path('../..', 'data', 'simulated_Hawkes', 'K3_C5')
ss, Ts, class2idx, _ = load_data(path, maxlen=-1, ext='csv', datetime=False)

gt_ids = pd.read_csv(Path(path, 'clusters.csv'))['cluster_id'].to_numpy()
gt_ids = torch.LongTensor(gt_ids)

In [4]:
N = len(ss)
D = 5
basis_fs = [lambda x: torch.exp(- x**2 / (3.*(k+1)**2) ) for k in range(D)]

hp = PointProcessStorage(ss, Ts, basis_fs)

C = len(class2idx)
K = 3

ntrials = 3
niter = 10

labels = torch.zeros(ntrials, len(ss))
nlls = torch.zeros(ntrials, niter)

for i in range(ntrials):
    Sigma = torch.rand(C, C, D, K)
    B = torch.rand(C, K)
    alpha = 1.

    model = DirichletMixtureModel(K, C, D, alpha, B, Sigma)
    EM = EM_clustering(hp, model)
    r, nll_history, r_history = EM.learn_hp(niter=niter, ninner=[2,3,4,5,6,7] + (niter - 6)*[8])

    labels[i] = r.argmax(-1)
    nlls[i] = torch.FloatTensor(nll_history)

    print(f'Purity: {purity(labels[i], gt_ids):.4f}')


1it [01:36, 96.71s/it]
NLL / N: 679.9168
2it [02:03, 75.61s/it]
NLL / N: -60.2518
3it [02:37, 63.36s/it]
NLL / N: -0.6981
4it [03:11, 54.53s/it]
NLL / N: 20.1141
5it [03:54, 50.84s/it]
NLL / N: 20.1951
6it [04:35, 48.15s/it]
NLL / N: 20.4390
7it [05:24, 48.29s/it]
NLL / N: 20.7928
8it [06:17, 49.75s/it]
NLL / N: 20.9816
9it [07:30, 56.76s/it]
NLL / N: 21.3264
10it [08:50, 63.62s/it]
NLL / N: 21.2528
11it [10:13, 69.36s/it]
NLL / N: 21.3383
12it [11:17, 67.84s/it]
NLL / N: 21.4435
13it [12:55, 76.87s/it]
NLL / N: 21.5729
14it [14:07, 75.39s/it]
NLL / N: 21.5506
15it [15:28, 77.27s/it]
NLL / N: 21.3079
16it [17:19, 87.37s/it]
NLL / N: 21.3306
17it [18:55, 89.80s/it]
NLL / N: 21.3939
18it [21:04, 101.70s/it]
NLL / N: 21.3338
19it [22:16, 92.67s/it] 
NLL / N: 21.5600
20it [23:47, 71.37s/it]
0it [00:00, ?it/s]
NLL / N: 21.3110
Purity: 0.9033
1it [01:43, 103.57s/it]
NLL / N: 665.3323
2it [02:11, 80.93s/it] 
NLL / N: -51.3180
3it [03:05, 72.69s/it]
NLL / N: -3.0699
4it [03:52, 65.07s/it]
NLL 

In [5]:
pur_val_mean = np.mean([purity(x, gt_ids) for x in labels])
pur_val_std = np.std([purity(x, gt_ids) for x in labels])

In [6]:
print(f'Purity: {pur_val_mean:.4f}+-{pur_val_std:.4f}')

Purity: 0.8433+-0.1039
